In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
from Network import Network
from GridSearch import GridSearcher
from activation_functions import sigmoid, relu,relu_derivative, sigmoid_derivative, tanh, tanh_derivative
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


In [22]:
# Set random seed for reproducibility
np.random.seed(42)
df_vote = pd.read_csv("CongressionalVotingID.shuf.lrn.csv", index_col="ID")
df_vote = df_vote.applymap(lambda x: 1 if x == "y" else 0 if x == "n" else x)
df_vote = df_vote.replace("democrat", 1)
df_vote = df_vote.replace("republican", 0)
df_vote = df_vote.replace("unknown", 0)

In [23]:
x_train, x_test, y_train, y_test = train_test_split(df_vote.iloc[:,1:], df_vote.iloc[:, 0],test_size=0.4, random_state=11)

In [10]:
# x_train = np.expand_dims(x_train.to_numpy(), axis=1)
# y_train = np.expand_dims(y_train.to_numpy(), axis=1)
# x_test = np.expand_dims(x_test.to_numpy(), axis=1)
# y_test = np.expand_dims(y_test.to_numpy(), axis=1)

In [24]:
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)
learning_rate=0.1

In [25]:
network = Network(learning_rate, epochs=1000, node_counts = [3, 3, 3],activation_function=(tanh, tanh_derivative))

In [26]:
params = {
    "learning_rate": [0.1, 0.2],
    "epochs": [10, 100],
    "node_counts": [  [3], [3, 3]],
    "activation_function":[(sigmoid,sigmoid_derivative)]
}
gs = GridSearcher(network, params, X_train, y_train)
gs_results = gs.search()
gs_results

learning_rate  epochs node_counts  \
0            0.1      10         [3]   
1            0.1      10      [3, 3]   
2            0.1     100         [3]   
3            0.1     100      [3, 3]   
4            0.2      10         [3]   
5            0.2      10      [3, 3]   
6            0.2     100         [3]   
7            0.2     100      [3, 3]   

                                 activation_function  \
0  (<function sigmoid at 0x0000020A6D187BA0>, <fu...   
1  (<function sigmoid at 0x0000020A6D187BA0>, <fu...   
2  (<function sigmoid at 0x0000020A6D187BA0>, <fu...   
3  (<function sigmoid at 0x0000020A6D187BA0>, <fu...   
4  (<function sigmoid at 0x0000020A6D187BA0>, <fu...   
5  (<function sigmoid at 0x0000020A6D187BA0>, <fu...   
6  (<function sigmoid at 0x0000020A6D187BA0>, <fu...   
7  (<function sigmoid at 0x0000020A6D187BA0>, <fu...   

                                          test_score  \
0  [0.29729729729729726, 0.2777777777777778, 0.21...   
1  [0.29729729729729726, 0.2777777777777778, 0.21...   
2  [0.29729729729729726, 0.2777777777777778, 0.21...   
3  [0.29729729729729726, 0.2777777777777778, 0.21...   
4  [0.29729729729729726, 0.2777777777777778, 0.21...   
5  [0.29729729729729726, 0.2777777777777778, 0.21...   
6  [0.29729729729729726, 0.2777777777777778, 0.21...   
7  [0.29729729729729726, 0.2777777777777778, 0.21...   

                                            fit_time  \
0  [0.3036165237426758, 0.315920352935791, 0.3599...   
1  [0.48787784576416016, 0.4678800106048584, 0.49...   
2  [3.6010384559631348, 3.4330811500549316, 3.617...   
3  [5.190312147140503, 5.098337888717651, 5.14632...   
4  [0.28392887115478516, 0.32391929626464844, 0.2...   
5  [0.4518866539001465, 0.43589091300964355, 0.43...   
6  [3.459127426147461, 3.4991159439086914, 3.4271...   
7  [5.059338331222534, 5.043345928192139, 5.13131...   

                                          score_time  score_mean  
0  [0.00400090217590332, 0.007999420166015625, 0....    0.244796  
1  [0.007998943328857422, 0.008001565933227539, 0...    0.244796  
2  [0.003999948501586914, 0.04398798942565918, 0....    0.244796  
3  [0.007999658584594727, 0.011994361877441406, 0...    0.244796  
4  [0.003999948501586914, 0.008002042770385742, 0...    0.244796  
5  [0.003998279571533203, 0.007999420166015625, 0...    0.244796  
6  [0.003998756408691406, 0.03599286079406738, 0....    0.244796  
7  [0.007997512817382812, 0.003996610641479492, 0...    0.244796

In [27]:
selected_prams=gs_results.nlargest(1,['score_mean'])
selected_prams

learning_rate  epochs node_counts  \
0            0.1      10         [3]   

                                 activation_function  \
0  (<function sigmoid at 0x0000020A6D187BA0>, <fu...   

                                          test_score  \
0  [0.29729729729729726, 0.2777777777777778, 0.21...   

                                            fit_time  \
0  [0.3036165237426758, 0.315920352935791, 0.3599...   

                                          score_time  score_mean  
0  [0.00400090217590332, 0.007999420166015625, 0....    0.244796

In [85]:
a=selected_prams["activation_function"]
type(a)
activation=[fun for fun in a.iloc[0]]
# print ([globals().get(fun.__name__) for fun in a.iloc[0]])
activation=tuple(activation)
print(activation)



(<function sigmoid at 0x0000020A6D187BA0>, <function sigmoid_derivative at 0x0000020A6D187C40>)


In [86]:
new_network= Network(learning_rate=selected_prams["learning_rate"], epochs=selected_prams["epochs"], node_counts=selected_prams["node_counts"],activation_function=activation)

In [ ]:
new_network.fit(x_train, y_train)

out = new_network.predict(x_test)

In [ ]:

# learning_rate = 0.1


# network1 = Network(learning_rate)
# input_values = 16
# output_values = 1
# intermediate_values = 3
# layers = 3
# epochs = 1000

# for j in range(layers):
#     i = intermediate_values
#     o = intermediate_values

#     if not j:
#         i = input_values
#     elif j + 1 == layers:
#         o = output_values

#     fc = FcLayer(i, o)
#     ac = AcLayer(tanh, tanh_prime)
#     network1.add(fc)
#     network1.add(ac)

# network1.train(x_train, y_train, epochs=epochs)

# out = network1.predict(x_train)

In [ ]:
# y_pred=network1.predict(x_test)
# y_pred=np.where(np.array(y_pred) > 0.5, 1, 0)
# np.unique(y_pred)

***Classification report for Self Neural Network***

In [ ]:
print(classification_report(y_pred=np.squeeze(y_pred),y_true=np.squeeze(y_test)))

***Tensor Flow***

In [281]:
def train_model(x_train,y_train, num_nodes, dropout_prob,lr, batch_size, epochs):
    nn_model=tf.keras.Sequential([tf.keras.layers.Dense(num_nodes, activation='relu', input_shape=(16,)),
                              tf.keras.layers.Dropout(dropout_prob),
                             tf.keras.layers.Dense(num_nodes, activation='relu'),
                             tf.keras.layers.Dropout(dropout_prob),
                             tf.keras.layers.Dense(1, activation='sigmoid')])

    nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss="binary_crossentropy",metrics=['accuracy'])

    #x_train=x_train.astype(float)

    history = nn_model.fit(x_train,y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
    return nn_model, history

In [282]:
nn_model, history= train_model(np.squeeze(x_train),np.squeeze(y_train),16 , 0,0.1, 32, 1000)

In [ ]:
y_pred=nn_model.predict(np.squeeze(x_test))
y_pred=(y_pred>0.5).astype(int).reshape(-1)
y_pred


print(classification_report(y_pred,np.squeeze(y_test)))